### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split


### Чтение файла

In [4]:
df = pd.read_csv('corp_lemm.csv')
df = df[df['text'].notnull()]
df = df.drop(['Unnamed: 0'], axis = 1)

In [5]:
df.head(5)

,text,rated,coloring,date,name,bank,sex,text_lemmatized
0,Возмутительный способ обмана придумали крючкот...,1,bad,"04 апр, 2022",Сергей,vtb,male,возмутительный способ обман придумать крючкотв...
1,"Не храните в нем деньги, ни коем случае! У мен...",1,bad,"30 мар, 2022",Альбина,uralsib,female,не хранить немой деньга кой случай у карта сч...
2,С 09.03.22 не могу получить средства с евро сч...,1,bad,"24 мар, 2022",Ирина,uralsib,female,с мочь получить средство евро счёт должный выд...
3,"Очень долгое обслуживание, сотрудники совершен...",1,bad,"14 мар, 2022",Александр,gazprombank,male,очень долгий обслуживание сотрудник совершенно...
4,Звоню третий день по вопросу валюты. Оператор ...,1,bad,"14 мар, 2022",Сергей Владимирович,gazprombank,unkn,звонить третий день вопрос валюта оператор го...


### Анализ тональности

In [6]:
#присвоение целевого признака
sentiment_mapping = {'bad': 0, 'neutral': 0, 'good': 1}
df['label'] = df['coloring'].map(sentiment_mapping)
df['label'].value_counts()

1    2120
0    1996
Name: label, dtype: int64

In [7]:
#разбиение на обучающую и тестовую выборки
train, test = train_test_split(df,
                        test_size = 0.2,
                        random_state = 12348,
                       )

print(train.shape)
print(test.shape)

(3292, 9)
(824, 9)


In [8]:
#распределение целевого признака
for sample in [train, test]:    
    print(sample[sample['label'] == 1].shape[0] / sample.shape[0])

0.5212636695018226
0.49029126213592233


Так как в модели логистической регрессии качество алгоритма было выше при векторизации текстов метрикой TF-IDF, эта метрика была также использована в дальнейшем исследовании.

In [9]:
#векторизация
tfidf = TfidfVectorizer(ngram_range=(1,1))

In [10]:
tfidf_train_lemma = tfidf.fit_transform(train['text_lemmatized'])

In [11]:
tfidf_test_lemma = tfidf.transform(test['text_lemmatized'])

In [12]:
tfidf_train_lemma.shape

(3292, 12705)

In [13]:
tfidf_test_lemma.shape

(824, 12705)

В Scikit-learn представлено три модели Наивного Байеса в зависимости от распределения данных: бинарного, дискретного и непрерывного. Так как распределение признаков в собранном нами корпусе дискретное, была выбрана модель MultinomialNB.

In [14]:
#подбор оптимального значения параметра alpha, который отвечает за сглаживание модели 
parameter = [1, 0, 0.1, 0.01, 0.001, 0.0001]
param_grid = {'alpha': parameter}
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5)
grid_search.fit(tfidf_train_lemma, train['label'])
Alpha, best_score = grid_search.best_params_, grid_search.best_score_

C:\Users\js\anaconda3\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
C:\Users\js\anaconda3\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
C:\Users\js\anaconda3\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
C:\Users\js\anaconda3\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
C:\Users\js\anaconda3\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, 

In [15]:
Alpha

{'alpha': 0.1}

In [16]:
best_score

0.9122110963004644

In [17]:
#обучение модели
model = MultinomialNB(0.1)
model.fit(tfidf_train_lemma, train['label'])
predicted = model.predict(tfidf_test_lemma)


C:\Users\js\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass alpha=0.1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [18]:
predictedtr =  model.predict(tfidf_train_lemma)

In [19]:
pred_train, pred_test = predictedtr, predicted

In [20]:
train.insert(0, 'predictions', [x.round(3).tolist() for x in pred_train])
test.insert(0, 'predictions', [x.round(3).tolist() for x in pred_test])
test['predictions'] = [x.round(3).tolist() for x in pred_test]


<ipython-input-20-9c7250abafcf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predictions'] = [x.round(3).tolist() for x in pred_test]


In [21]:
pred = []
for i in test['predictions']:
    if i==0:
        pred.append('neg')
    else:
        pred.append('pos')
tr = []
for i in test['rated']:
    if i == 5:
        tr.append('pos')
    else:
        tr.append('neg')
test = test.drop(['coloring', 'text_lemmatized','date','name','bank','sex'], axis=1)
test['true'] = tr
test['predicted'] = pred

In [22]:
test.head(5)

,predictions,text,rated,label,true,predicted
2303,1,"Обратился По вопросу обслуживания карты,сотруд...",4,1,neg,pos
1184,0,мая была подана заявка в банк для утверждения ...,2,0,neg,neg
1100,0,ноября в 14. 10 села в ожидании очереди в касс...,2,0,neg,neg
2317,1,"Хотела закрыть карту,но уговорили оставить. До...",4,1,neg,pos
2397,1,Оформляла продажу квартиры и нужна была банков...,4,1,neg,pos


### Оценка качества работы алгоритма

In [23]:
a = []
b = []
for i in test['predicted']:
    a.append(i)
for i in test['true']:
    b.append(i)

In [24]:
tp = 0 
tn = 0 
fp = 0 
fn = 0
t_f = []
for i in range(0, len(a)):
    if a[i] == b[i]:
        if a[i] == 'pos':
            tp+=1
            t_f.append('tp')
        else:
            tn+=1
            t_f.append('tn')
    elif a[i] != b[i]:
        if a[i] == 'pos':
            fp+=1
            t_f.append('fp')
        else:
            fn+=1
            t_f.append('fn')
        

In [25]:
test['result'] = t_f

In [26]:
accuracy = (tp+tn)/len(test)
precision = tp/ (tp+fp)
recall = tp/(tp+fn)
f1 = 2/(1/precision+1/recall)
f_measure = (2*precision*recall)/(precision+recall)

In [27]:
print('accuracy', accuracy)
print('precision', precision)
print('recall', recall)
print('f1-score', f1)
print('f-measure', f_measure)

accuracy 0.8422330097087378
precision 0.6638888888888889
recall 0.9637096774193549
f1-score 0.7861842105263157
f-measure 0.7861842105263158


In [28]:
t_res = test.query('result == "tn" or result == "tp"')
f_res = test.query('result == "fn" or result == "fp"')
pd.options.display.max_colwidth = 10000

In [29]:
f_res.query('result == "fn"')  #ложноотрицательные отзывы

,predictions,text,rated,label,true,predicted,result
3630,0,"Пополняла счёт Халвы в устройстве самообслуживания. Устройство одно не работала, сотрудник предложила воспользоваться другим или проехать в офис с кассой",5,1,pos,neg,fn
2868,0,"Мне до 10 числа нужно было внести платеж по ипотеке и вспомнила я об этом только вечером после работы, когда мой телефон успешно разрядился. Ну благо по пути с работы есть отделение гпб, куда я и отправилась. В физических отделениях я давно не была и ожидала злых теток и очередей, но все оказалось куда цивильнее. При чем на столько, когда мою проблему услышали. Мне подсказали как я могу внести платеж через кассу и предложили как альтернативу зарядить телефон на стойке и сдлать это через приложение. Второй вариант меня пряитно шокировал - его я и выбрала. Спасибо милому администратору за помощь.",5,1,pos,neg,fn
2882,0,"Оказывается, можно легко и спокойно застраховать карту, так точно можно не бояться нападения мошенников, можно застраховать жизнь и здоровье, спокойно можно получить доступ к программе лечения критических заболеваний у врачей России и зарубежом. Страхование детей и так далее, очень большой спектр страхования для любых целей. Балдеж. Буду страховаться и страховать то, что дорого.",5,1,pos,neg,fn
2867,0,"История банальная и случалась со мной уже много раз - банкомат не вернул карту. Но концовка этой истории в этот раз отличалась. На новом месте работы обслуживающий банк - Газпромбанк, собственно их карта у меня и появилась. Вполне удобно, учитывая, что отделение через дорогу от работы. На обеде в кафешке огорчили новостью, что терминал не работает и я в пошла в отделение, чтобы снять деньги. Там банкомат меня и подвел. Думала, что обед отменяется, но ко мне подошла менеджер, которая увидела мою ситуацию и помогла с решением. Я оставила заявку на полноценную карту, а пока взяла моментальную на тот же счет и сняла наличные. В итоге даже успела пообедать.",5,1,pos,neg,fn
4104,0,Потеряла карту Халва и обратилась в банк по поводу блокировки старой карты и выправления новой. Сотрудники сразу же решили мою проблему.,5,1,pos,neg,fn
2833,0,"Так вышло, что заявка моя оставалась незамеченной порядка двух недель, но после звонка в поддержку процесс сдвинулся с мертвой точки и дальше проблем не было. Самим кредитом и условиями я доволен + удобно, что все привозят на дом курьером.",5,1,pos,neg,fn
3252,0,"Я являюсь муниципальным служащим. Для получения справки из банка затрат ла минут 5 личного времени. Очень хорошо все организовано, не то что в других банках, а мне есть с чем сравнить. Спасибо вам что цените чужое время",5,1,pos,neg,fn
4041,0,"Позвонила,указала проблему,решили,позвонили предупредили,предложили повысить кредит,в общем все понравилось.",5,1,pos,neg,fn
2798,0,"Взял кредит наличными В Газпромбанке, думаю что ставка сейчас там не заоблачная в текущих условиях. На сайте заполнил простую анкету: ФИО, номер, почта и прочитать соглашения. Через несколько часов мне позонил специалист, извинился за о ожидание, хотя я не злился даже, и сказал, что мой кредит предварительно одобрен. Сказал, что я могу подъезжать в отделение. Я приехал на следующий день, приехал специально рано, к открытию, чтобы не стоять в очереди. Но не один я такой хитрый, человек 5-7 уже стояло, но сотрудники оперативно сработали. Менеджер ответил на вопросы и рассказал, как и когда погашать кредит. Девушка Марина была вежлива, внимательна и доходчиво все объяняла, видн, что знает свое дело. Кредит я получил, благдарю",5,1,pos,neg,fn


In [30]:
f_res.query('result == "fp"') #ложноположительные отзывы

,predictions,text,rated,label,true,predicted,result
2303,1,"Обратился По вопросу обслуживания карты,сотрудник бала вежлива,внимательна,все квалифицировано объяснила.",4,1,neg,pos,fp
2317,1,"Хотела закрыть карту,но уговорили оставить. До сих пор пользуюсь, пользуюсь уже где-то 4 года, нравится получать кешбек",4,1,neg,pos,fp
2397,1,"Оформляла продажу квартиры и нужна была банковская ячейка для передачи денег. На Обводном ближайшим был Энергомашбанк. Сотрудницы банка Марина и Елена, если не ошибаюсь, очень быстро провели сделку, остались довольны и мы и продавцы квартиры. Сроки поджимали, а для нас было важно быстрое оформление документов. Все сделали оперативно, легко и вежливо. Хотя покупатели наши немного нервничали. Но все прошло отлично. Спасибо все, пошли отмечать=)",4,1,neg,pos,fp
2727,1,"Без обмана и купюры как новые,чистенькие,не стыдно расплачиваться ими,кассиры приятные на внешность,одеты в рубашечки,бейджики,как в сбербанке прям)))Вообщем по сравнению с конторами которые обманывают на Арбате,постоянно кидая,эта операционка чистая,сайт конечно бы получше им сделать,а так желаю только дальнейшего развития и поменьше инциндентов.",4,1,neg,pos,fp
2256,1,"Неоднократно обращалась за консультация и к сотрудникам офиса Банка, расположенного по ул. Ленина в г. Рузаевка я Республика Мордовия. Сотрудники всегда приветливые, внимательные, стараются оперативно разобраться в сути вопроса и подсказывают возможные варианты положительного решения.",4,1,neg,pos,fp
...,...,...,...,...,...,...,...
353,1,"Грамотного и провесианалов один сотрудник на банк. Был в двух офисах в городе. Одно и тоже очередь, долгое ожидание. В обоих банках сотрудники консультируються у одной. Которая ( впечетление такое) и осущевсляет всю работу банка.",1,0,neg,pos,fp
1625,1,"Здравствуйте. Оформлял карту халва по акции приведи друга, мне одобрили только дебетовую карту. Далее в банке на мой вопрос про вклад, сотрудник сказала о, у нас сейчас если кладёте на вклад от 100.000₽ вам одобрят кредитный лимит. Я пошёл в банк где у меня лежал вклад, снял все, положил к вам на вклад. По новой оформил заявку по акции приведи друга и что? Мне опять одобрили только дебетовую карту. Вопрос, зачем я бегал снимал вклад с одного банка, ложил в другой, если ничего не изменилось? Что тяжело было одобрить даже какой-нибудь минимальный лимит в размере 5000₽ (и это при вкладке больше полумиллиона) , чтобы я и мой друг смогли воспользоваться, акцией?",3,0,neg,pos,fp
2176,1,"Много хороших моментов, так же и плохих. Порог кеш бека все портит, совершали бы много покупок по вашей карте, но увы. Порог кеш бэка",4,1,neg,pos,fp
2655,1,"Итак, плюсы. Очень быстро оформили карту. Никаких проблем при этом не возникло, но у меня до этого был всего один небольшой потребительский кредит, который погашен досрочно, поэтому ничего удивительного в этом нет. Картой пользовалась (и пользуюсь) очень активно, большей частью при совершении покупок через интернет. Очень удобное приложение для интернет-банкинга на Android, правда, старое было намного лучше. Очень порадовало, что даже в другом городе мне в офисе сменили привязанный номер телефона и приняли годовые взносы, не заставив ехать за 350 километров :) Ну и напоследок несколько минусов. Как-то мне позвонила милая девочка и предложила увеличить кредитный лимит. На мой вопрос ""что от меня для этого нужно"" начала уточнять личные данные и сказала подойти в офис... где уже не такая вежливая сотрудница начала ещё раз задавать те же самые вопросы (зачем тогда меня держали на телефоне 40 минут?), а потом потребовала справку с работы о доходах. на резонное возмущение, что я бы вообще тогда не затевала это, если бы мне рассказали все требования по телефону, мне резко ответили ""по телефону мы только предлагаем услуги, а решение принимаю Я"". Когда эпопея с увеличением кредитного лимита растянулась ещё сильнее (после того, как я принесла требуемую справку, девушка ушла в отпуск), я просто махнула рукой, так как изначально экстренной необходимости в этом воо

### Общие выводы

Видим, что точность у Байесовского классификатора больше, чем у модели логистической регрессии. Полнота немного меньше, это означает, что среди ложно предсказанных результатов чаще встречаются ложно негативные. Проанализировав отзывы, определенные как ложно негативные, мы обнаружили, что большинство из них имеет оценку ‘4’, при этом сами комментарии, действительно, скорее негативные и модель хорошо справляется с задачей.